In [1]:
!pip install trax


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires pr

In [2]:
import trax
import trax.layers as tl
import trax.fastmath.numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import random as rnd
from trax.supervised.training import TrainTask,EvalTask ,Loop

/opt/conda/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


In [3]:
data=[]
with open('/kaggle/input/shakespeare-plays/alllines.txt') as lines:
    for line in lines :
        data.append(line.strip())
        
data[:10]    

['"ACT I"',
 '"SCENE I. London. The palace."',
 '"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"',
 '"So shaken as we are, so wan with care,"',
 '"Find we a time for frighted peace to pant,"',
 '"And breathe short-winded accents of new broils"',
 '"To be commenced in strands afar remote."',
 '"No more the thirsty entrance of this soil"',
 '"Shall daub her lips with her own children\'s blood,"',
 '"Nor more shall trenching war channel her fields,"']

In [4]:
def tokenize_data(data):
    clean_data=[]
    for line in data:
        words=[]
        for char in line:
            if char not in punctuation:
                words.append(char.lower())
            
        clean_data.append(words)
    return(clean_data)

In [5]:
clean_data=tokenize_data(data)
clean_data[0]

['a', 'c', 't', ' ', 'i']

In [6]:
def get_vocab(data):
    vocab={'<pad>':0,'<eos>':1}
    for line in data:
        for charachter in line:
            if charachter not in vocab:
                vocab[charachter]=ord(charachter.lower())
    return vocab
vocab=get_vocab(data)
len(vocab)

79

In [7]:
def get_tensor(data,vocab):
    tensor=[]
    for word in data :
        tensor.append(vocab[word])
    return(tensor+[vocab['<eos>']])

In [8]:
print(len(get_tensor(clean_data[2],vocab)))
len(clean_data[2])

93


92

In [9]:
clean_data=[i for i in clean_data if len(i)<=63]


In [10]:
len(clean_data)

111036

In [11]:
def generator(data,vocab,batch_size=256,shuffle=True):
    data_size=len(data)
    index=[*range(data_size)]
    Input=[]
    batch=[]
    Mask=[]
    idx=0
    while True:
        if idx>=data_size:
            idx=0
        if shuffle:
            rnd.shuffle(index)
        

        tensor=get_tensor(data[index[idx]],vocab)
        batch.append(tensor)
        idx+=1
        if len(batch)==batch_size:
            max_len=max([len(i)for i in batch])
            max_len=int(2**np.ceil(np.log2(max_len)))
            for tensor in batch:
                tensor=tensor+([vocab['<pad>']]*(max_len-len(tensor)))
                Input.append(tensor)
            Input=np.array(Input)
            yield ((Input,Input))
            Input,batch=[],[]




In [12]:
X,Y=next(generator(clean_data,vocab,64))
print('Shape of Input: ',X.shape)
print('Shape of Output: ',Y.shape)

Shape of Input:  (64, 64)
Shape of Output:  (64, 64)


In [13]:
train_size=int(len(clean_data)*.8)
val_size=int(len(clean_data)*.9)

train_data=clean_data[:train_size]
val_data=clean_data[train_size:val_size]
test_data=clean_data[val_size:]
print(len(train_data))
print(len(val_data))
print(len(test_data))

88828
11104
11104


In [14]:
batch_size=512
train_generator=trax.data.add_loss_weights(generator(clean_data,vocab,batch_size),vocab['<pad>'])
val_generator=trax.data.add_loss_weights(generator(clean_data,vocab,batch_size),vocab['<pad>'])

In [15]:
x,y,m=next(train_generator)
print(x.shape,y.shape,m.shape)

(512, 64) (512, 64) (512, 64)


In [16]:
# GRADED FUNCTION: GRULM
def model(vocab, d_model=512, n_layers=2, mode='train'):
    vocab_size=len(vocab)
    model = tl.Serial( 
      tl.ShiftRight(mode='train'), # Stack the ShiftRight layer
      tl.Embedding(vocab_size,d_model), # Stack the embedding layer
      [tl.GRU(d_model) for _ in range(n_layers)], # Stack GRU layers of d_model units keeping n_layer parameter in mind (use list comprehension syntax)
      tl.Dense(vocab_size), # Dense layer
      tl.LogSoftmax(), # Log Softmax
    ) 
    ### END CODE HERE ###
    return model

In [17]:
len(vocab)

79

In [18]:
print(model(vocab,512,4))

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_79_512
  GRU_512
  GRU_512
  GRU_512
  GRU_512
  Dense_79
  LogSoftmax
]


In [19]:
# PLEASE, DO NOT MODIFY OR DELETE THIS CELL
from trax.supervised import training
import itertools
def get_train_eval_tasks(train_generator,val_generator):
    
    rnd.seed(271)

    train_task = training.TrainTask(
        labeled_data=train_generator,
        loss_layer=tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(0.0001),
        n_steps_per_checkpoint=10,
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,        
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    )
    
    return train_task, eval_task
    

train_task, eval_task = get_train_eval_tasks(train_generator,val_generator)


In [20]:
dir_path = './model/'

n_step=10

def trainig (classifier,n_step,train_task,eval_task,dir_path):
    train=training.Loop(classifier,train_task,eval_tasks=[eval_task], output_dir=dir_path)
    train.run(n_step)
    return train

In [21]:
num_class=len(vocab)
n_dimension=512
classifier = model(vocab,n_dimension)
print('Show Sup_Layer in My Model\n\n',classifier)

Show Sup_Layer in My Model

 Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_79_512
  GRU_512
  GRU_512
  Dense_79
  LogSoftmax
]


In [22]:
mo=trainig (classifier,n_step,train_task,eval_task,'./model/')

/opt/conda/lib/python3.7/site-packages/jax/_src/lib/xla_bridge.py:554: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "



Step      1: Total number of trainable weights: 3229775
Step      1: Ran 1 train steps in 58.09 secs
Step      1: train CrossEntropyLoss |  0.85439074
Step      1: eval  CrossEntropyLoss |  0.85988170
Step      1: eval          Accuracy |  0.17020158

Step     10: Ran 9 train steps in 415.13 secs
Step     10: train CrossEntropyLoss |  0.84690642
Step     10: eval  CrossEntropyLoss |  0.82580876
Step     10: eval          Accuracy |  0.16972932


In [24]:
x,y,m=next(train_generator)

In [39]:
32768/3222

10.17008069522036

## 32768

In [38]:
predicrion=mo.eval_model(x)

In [36]:
np.sum(np.argmax(predicrion,axis=2)==x)

DeviceArray(3222, dtype=int32)

In [ ]:
def accuracy(x_test,model)

In [33]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = './'
remove_folder_contents(folder_path)
os.rmdir(r'./')


OSError: [Errno 22] Invalid argument: './'